## Optimization packages playground

In [1]:
# Define f
a = 1.0
b = 100.0

"Rossenbrok's function"
rosenbrock(x::Vector{<:Real}, a::Real, b::Real) =  (a - x[1])^2 + b * (x[2] - x[1]^2)^2

"Closure"
f = x -> rosenbrock(x, a, b);

"Rossenbrok's function gradient"
function g!(G, x)
    G[1] = -2.0 * (1.0 - x[1]) - 400.0 * (x[2] - x[1]^2) * x[1]
    G[2] = 200.0 * (x[2] - x[1]^2)
end

"Rossenbrok's Hessian matrix"
function 𝐻rosenbrock!(H, x)
    H[1, 1] = 2.0 - 400.0 * x[2] + 1200.0 * x[1]^2
    H[1, 2] = -400.0 * x[1]
    H[2, 1] = -400.0 * x[1]
    H[2, 2] = 200.0
end;
    
"Analytic solution"
xₘᵢₙ = [a, a^2];


In [2]:
using Plots; pyplot()
x  = [-2:0.05:2;]
y = [-1:0.05:3;]
z = [f([x,y]) for x in x, y in y]

minZ = minimum(z[:]);  
maxZ = maximum(z[:]);

COL = append!([colorant"blue",colorant"lime"],range(colorant"yellow",colorant"red",length=20))
c =  minZ .+ (maxZ-minZ).*log.(1 .+z .- minZ) ./ log(1+maxZ-minZ)

Plots.plot(x,y,z,st=:surface,color=cgrad(COL,scale=:exp),#cgrad(:jet,c),
xlabel = "x",ylabel="y",zlabel="f(x,y)",zguidefontrotation=45,camera=(-30,30))

ERROR: LoadError: InitError: 

could not load library "/home/mvanzulli/.julia/artifacts/b409c0eafb4254a980f9e730f6fbe56867890f6a/lib/libavdevice.so"
libssl.so.1.1: cannot open shared object file: No such file or directory
Stacktrace:
 [1] macro expansion
   @ ~/.julia/packages/JLLWrappers/QpMQW/src/products/library_generators.jl:54 [inlined]
 [2] __init__()
   @ FFMPEG_jll ~/.julia/packages/FFMPEG_jll/OCtN5/src/wrappers/x86_64-linux-gnu.jl:39
 [3] top-level scope
   @ stdin:1
during initialization of module FFMPEG_jll
in expression starting at /home/mvanzulli/.julia/packages/FFMPEG/OUpap/src/FFMPEG.jl:1
in expression starting at stdin:1


ERROR: LoadError: 

Failed to precompile FFMPEG [c87230d0-a227-11e9-1b43-d7ebe4e7570a] to /home/mvanzulli/.julia/compiled/v1.8/FFMPEG/jl_iA7KnG.
Stacktrace:
 [1] include(x::String)
   @ Plots ~/.julia/packages/Plots/tDHxD/src/Plots.jl:1
 [2] top-level scope
   @ ~/.julia/packages/Plots/tDHxD/src/Plots.jl:168
 [3] top-level scope
   @ stdin:1
in expression starting at /home/mvanzulli/.julia/packages/Plots/tDHxD/src/animation.jl:1
in expression starting at /home/mvanzulli/.julia/packages/Plots/tDHxD/src/Plots.jl:1
in expression starting at stdin:1


ErrorException: Failed to precompile Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80] to /home/mvanzulli/.julia/compiled/v1.8/Plots/jl_QmOxMH.

### Optim.jl: Gradient required

In [5]:
using Optim, LineSearches

# Box 
lower = [0.3, -2.1]
upper = [30.0, 40.0]

# Initial guess
x₀ = [0.4,0.5];

### Available algos

In [6]:
inner_optimizer = GradientDescent()
inner_optimizer = ConjugateGradient()
momentum  = 10
inner_optimizer = LBFGS(m = momentum)

LBFGS{Nothing, InitialStatic{Float64}, HagerZhang{Float64, Base.RefValue{Bool}}, Optim.var"#20#22"}(10, InitialStatic{Float64}
  alpha: Float64 1.0
  scaled: Bool false
, HagerZhang{Float64, Base.RefValue{Bool}}
  delta: Float64 0.1
  sigma: Float64 0.9
  alphamax: Float64 Inf
  rho: Float64 5.0
  epsilon: Float64 1.0e-6
  gamma: Float64 0.66
  linesearchmax: Int64 50
  psi3: Float64 0.1
  display: Int64 0
  mayterminate: Base.RefValue{Bool}
, nothing, Optim.var"#20#22"(), Flat(), true)

In [24]:
# Set tolerances
#Ncalls to f 
outer_iters = 40
#Ncalls to alg step
iters = 30
# gradient tolerance 
GTOL = 1e-12
options = Optim.Options(g_tol = 1e-12, outer_iterations = outer_iters, iterations = iters, store_trace = true)

# Forward Diff
res_fdif = optimize(f, lower, upper, x₀, Fminbox(inner_optimizer),options, autodiff = :finite )
res = optimize(f, g!, lower, upper, x₀, Fminbox(inner_optimizer))

function show_results(res)
    @show summary(res)
    @show minimum(res)
    @show Optim.minimizer(res)
    @show Optim.iterations(res)
    @show Optim.iteration_limit_reached(res)
    # @show length(Optim.f_trace(res))
    @show Optim.f_calls(res)
    @show Optim.converged(res)
end

println("-------ForwardDiff with ∇--------")
show_results(res_fdif)

println("-------Grdient--------")
show_results(res)

-------ForwardDiff with ∇--------
summary(res) = "Fminbox with L-BFGS"
minimum(res) = 5.361093300320716e-17
Optim.minimizer(res) = [0.9999999926780513, 0.9999999853561207]
Optim.iterations(res) = 4
Optim.iteration_limit_reached(res) = false
Optim.f_calls(res) = 78
Optim.converged(res) = true
-------Grdient--------
summary(res) = "Fminbox with L-BFGS"
minimum(res) = 1.383848755571263e-22
Optim.minimizer(res) = [1.0000000000117624, 1.0000000000235425]
Optim.iterations(res) = 4
Optim.iteration_limit_reached(res) = false
Optim.f_calls(res) = 74
Optim.converged(res) = true


true

### Optim.jl: Gradient free 

In [25]:
inner_optimizer = NelderMead()

NelderMead{Optim.AffineSimplexer, Optim.AdaptiveParameters}(Optim.AffineSimplexer(0.025, 0.5), Optim.AdaptiveParameters(1.0, 1.0, 0.75, 1.0))

In [30]:
res_nmead = optimize(f, lower, upper, x₀, Fminbox(inner_optimizer),options)


println("-------Nelder Mead--------")
show_results(res_nmead)

-------Nelder Mead--------
summary(res) = "Fminbox with Nelder-Mead"
minimum(res) = 3.0143331585332986e-9
Optim.minimizer(res) = [0.9999499855832705, 0.9999022383782905]
Optim.iterations(res) = 5
Optim.iteration_limit_reached(res) = true
Optim.f_calls(res) = 350
Optim.converged(res) = true


true

## Black Box Optim

In [15]:
using BlackBoxOptim


method = :probabilistic_descent
method = :de_rand_1_bin_radiuslimited
method = :adaptive_de_rand_1_bin_radiuslimited

good_guess = [3.0, 7.2]
two_good_guesses = [[3.0, 7.2], [3.0, 7.2]]
res = bboptimize(f, two_good_guesses; 
SearchRange = [(lower[1], upper[1]), (lower[2], upper[2])], 
# NumDimensions = 2,
 Method = method,
 MaxTime = 1e-3,
 MaxFuncEvals = 2,
 )
# Maximum time allowed in seconds. 

# Number of function evaluations allowed, this needs  MaxTime to be false then the MaxFuncEvals plays a role

# Access to the solution 
best_candidate(res)
best_fitness(res)


Starting optimization with optimizer DiffEvoOpt{FitPopulation{Float64}, RadiusLimitedSelector, BlackBoxOptim.AdaptiveDiffEvoRandBin{3}, RandomBound{ContinuousRectSearchSpace}}
0.00 secs, 0 evals, 0 steps

Optimization stopped after 857 steps and 0.00 seconds
Termination reason: Max time (0.001 s) reached
Steps per second = 841807.62
Function evals per second = 946910.79
Improvements/step = Inf
Total function evaluations = 964


Best candidate found: [1.22183, 1.4815]

Fitness: 0.062106662



0.06210666187066345